![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FApplied+GenAI%2FEvaluation&file=Optimize+Prompts+Using+Evaluation+Metrics.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Evaluation/Optimize%20Prompts%20Using%20Evaluation%20Metrics.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FApplied%2520GenAI%2FEvaluation%2FOptimize%2520Prompts%2520Using%2520Evaluation%2520Metrics.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Evaluation/Optimize%20Prompts%20Using%20Evaluation%20Metrics.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Applied%20GenAI/Evaluation/Optimize%20Prompts%20Using%20Evaluation%20Metrics.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Optimize Prompts Using Evaluation Metrics

Prompt optimization rewrites system instructions to optimize the performance of a set of prompts on one or more evaluation metrics.

First, it is helpful to understand the Vertex AI GenAI evaluation service as covered in this workflow: [Evaluation For GenAI](./Evaluation%20For%20GenAI.ipynb). Evaluation is the comparison of a model's output to a baseline or ground truth using a metric to quantify the performance. Vertex AI offers pointwise metrics, pairwise metrics, and computed metrics for GenAI evaluation. These same evaluations can be used as the optimization goal for prompt optimization - the focus of this workflow.

The Vertex AI Prompt Optimization service is a tool that optimizes system instructions for a set of prompts. It offers modes for optimizing the system instructions with and without the inclusion of demonstrations, also known as multi-shot prompting. The content of this workflow is summarized here with direct links to official documentation:

- Documentation Link: [Optimize Prompts](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/prompt-optimizer)
- This service provides code: [GitHub Link to .py file](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/prompt_optimizer/vapo_lib.py)
    - And [example notebooks](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/gemini/prompts/prompt_optimizer)
- The user uses the code to initialize a prompt optimization job, which runs as a Vertex AI Custom Training Job
- The inputs are:
    - The current **System Instructions** - [Documentation Link](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/prompt-optimizer#template-si)
    - The **Prompt Template** for the sample prompts - [Documentation Link](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/prompt-optimizer#template-si)
        - If you have ground truth responses, or responses from a known good model, then the template can include a `{target}` variable that maps to the `target` values in the input file
        - If you don't have these responses, you can provide the `source_model` parameter in the configuration, and the optimization job will use the model to generate responses for comparison.
    - A **file of input data** for each sample prompt to be used with the prompt template - [Documentation Link](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/prompt-optimizer#prepare-sample-prompts)
        - Either a JSONL file or a CSV stored in a GCS bucket
    - **Configuration parameters** for the job - [Documentation Link](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/prompt-optimizer#configuration)
        - Which metrics to use, including custom metrics
        - Which target and source LLM to use
        - Many more optional parameters
- The outputs are:
    - Optimized System Instructions
    - Results for each evaluation step taken

---
## Colab Setup

To run this notebook in Colab run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
    print('Colab authorized to GCP')
except Exception:
    print('Not a Colab Environment')
    pass

Not a Colab Environment


---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform', '1.78.0'),
    ('google.cloud.storage', 'google-cloud-storage'),
    ('pandas', 'pandas')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [4]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [5]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

inputs:

In [6]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [7]:
REGION = 'us-central1'
SERIES = 'applied-genai'
EXPERIMENT = 'prompt-optimization'

BUCKET = PROJECT_ID # change to Bucket name if not the same as the Project ID

packages:

In [8]:
# Python standard library imports:
import json, io, requests, sys, types, datetime, time

# package imports
from IPython.display import Markdown, HTML, display

# vertex ai imports
from google.cloud import aiplatform
from google.cloud import storage
import vertexai

In [9]:
aiplatform.__version__

'1.78.0'

clients:

In [10]:
vertexai.init(project = PROJECT_ID, location = REGION)
gcs = storage.Client(project = PROJECT_ID)

---
## Optimize Prompts: Prepare Inputs

### Load The Code

The code is on [GitHub](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/prompt_optimizer/vapo_lib.py) as a `.py` file that is loaded in this session as a module with name `prompt_opt` by the following cell:

In [11]:
module_name = 'vapo_lib'
url = 'https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/prompts/prompt_optimizer/vapo_lib.py'
response = requests.get(url)
vapo_lib = types.ModuleType(module_name)
vapo_lib.__file__ = f'<remote>/{module_name}.py'
sys.modules[module_name] = vapo_lib
exec(response.text, vapo_lib.__dict__)

2025-01-29 23:54:44.195123: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738194884.222224 3337070 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738194884.230083 3337070 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-29 23:54:44.257623: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
import vapo_lib as prompt_opt

### Define Inputs

Define the current version of the system instruction and the template used for the prompt samples.

> **Details About Prompt Templates**
>
> A prompt template contains the structure of the prompt with the parts that change replaced by variables in the format `{variable_name}`.  The variable names cannot contain spaces.
>
> If a prompt contains multimodal content then the variable should be followed with the `MIME_TYPE` like this: `{image_variable} @@@image/jpeg`.

In [20]:
SYSTEM_INSTRUCTIONS = "Write poems."

In [21]:
PROMPT_TEMPLATE = "Write a {type} about {topic}."

### Prepare Data For Prompts

The prompt templates variables are read from a data struture provided as either a CSV or JSONL stored in a GCS bucket.  Here the data is first prepared as a list of dictionaries where each dictionary contains the key:value pairs for an instance of the prompt.

> **Details About Multimodal Content**
>
> If the prompt template includes multimodal content then the variable should get values that are the GCS location of the content to load.  Example: `{"image_variable": "gs://bucket_name/path/to/file/filename.extension"}`

In [22]:
prompt_parameters = [
    dict(type = 'Haiku', topic = 'Lego'),
    dict(type = 'Sonnet', topic = 'Lego'),
    dict(type = 'Limerick', topic = 'Lego'),
    dict(type = 'Acrostic', topic = 'Lego'),
    dict(type = 'Ode', topic = 'Lego')
]

### Store Prompt Data In GCS

This example converts the list of dictionaries to JSON lines and stores it directly in a GCS bucket as `prompt_parameters.jsonl`.

In [23]:
bucket = gcs.bucket(BUCKET)
blob = bucket.blob(f'{SERIES}/{EXPERIMENT}/prompt_parameters.jsonl')
with io.StringIO() as jsonl_file:
    for item in prompt_parameters:
        json.dump(item, jsonl_file)
        jsonl_file.write('\n')
    blob.upload_from_string(jsonl_file.getvalue(), content_type = 'application/jsonl')

Review the contents here:

In [64]:
print(blob.download_as_string().decode('utf-8'))

{"type": "Haiku", "topic": "Lego"}
{"type": "Sonnet", "topic": "Lego"}
{"type": "Limerick", "topic": "Lego"}
{"type": "Acrostic", "topic": "Lego"}
{"type": "Ode", "topic": "Lego"}



### Define Inputs Parameters And Validate Data

The code contains methods for determining the required structure of the inputs and the completness of the inputs.

In [24]:
SOURCE_MODEL = "gemini-1.5-flash-001" # or provide ground truth
TARGET_MODEL = "gemini-1.5-flash-002" # the model for which the optimized system instructions are created for
EVAL_METRICS = ['coherence', 'fluency'] # https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/prompt-optimizer#supported-evaluation-metrics

In [25]:
prompt_opt.is_run_target_required(
    eval_metric_types = EVAL_METRICS,
    source_model = SOURCE_MODEL
)

False

In [26]:
prompt_opt.validate_prompt_and_data(
    template = '\n'.join([SYSTEM_INSTRUCTIONS, PROMPT_TEMPLATE]),
    dataset_path = f'gs://{bucket.name}/{blob.name}',
    placeholder_to_content = '{}',
    label_enforced = prompt_opt.is_run_target_required(
        eval_metric_types = EVAL_METRICS,
        source_model = SOURCE_MODEL
    )
)

---
## Optimize Prompts: System Instructions

### Set Optimization Mode

In [ ]:
OPTIMIZATION_MODE = "instruction" # choices are instuction, demonstration, instruction_and_demo

### Run Optimization Job On Vertex AI Training

In [27]:
job_name = 'lego_lyrics_' + datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
vertex_job = prompt_opt.run_apd(
    config = dict(
        project = PROJECT_ID,
        system_instruction = SYSTEM_INSTRUCTIONS,
        prompt_template = PROMPT_TEMPLATE,
        target_model = TARGET_MODEL,
        target_model_location = REGION, 
        eval_metrics_types = EVAL_METRICS,
        eval_metrics_weights = [.5, .5],
        aggregation_type = 'weighted_sum',
        source_model = SOURCE_MODEL,
        optimization_mode = OPTIMIZATION_MODE,
        input_data_path = f'gs://{bucket.name}/{blob.name}',
        output_path = f'gs://{bucket.name}/{SERIES}/{EXPERIMENT}/{job_name}'
    ),
    bucket_uri = f'gs://{bucket.name}/{SERIES}/{EXPERIMENT}/{job_name}',
    display_name = job_name
)



Job display name: lego_lyrics_2025-01-29T23:55:21
Creating CustomJob
CustomJob created. Resource name: projects/1026793852137/locations/us-central1/customJobs/1425355712598376448
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/1026793852137/locations/us-central1/customJobs/1425355712598376448')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/1425355712598376448?project=1026793852137


Check out the Vertex AI Training Job with the link in the output above.

In [28]:
vertex_job.name, vertex_job.display_name, vertex_job.resource_name

('1425355712598376448',
 'lego_lyrics_2025-01-29T23:55:21',
 'projects/1026793852137/locations/us-central1/customJobs/1425355712598376448')

In [29]:
while vertex_job.state == aiplatform.gapic.JobState.JOB_STATE_PENDING or vertex_job.state == aiplatform.gapic.JobState.JOB_STATE_RUNNING:
    print(f"Job state: {vertex_job.state.name}, checking again in 30 seconds...")
    time.sleep(30)

Job state: JOB_STATE_PENDING, checking again in 30 seconds...
Job state: JOB_STATE_PENDING, checking again in 30 seconds...
Job state: JOB_STATE_PENDING, checking again in 30 seconds...
Job state: JOB_STATE_PENDING, checking again in 30 seconds...
Job state: JOB_STATE_PENDING, checking again in 30 seconds...
Job state: JOB_STATE_PENDING, checking again in 30 seconds...
Job state: JOB_STATE_RUNNING, checking again in 30 seconds...
Job state: JOB_STATE_RUNNING, checking again in 30 seconds...
Job state: JOB_STATE_RUNNING, checking again in 30 seconds...
Job state: JOB_STATE_RUNNING, checking again in 30 seconds...
Job state: JOB_STATE_RUNNING, checking again in 30 seconds...
Job state: JOB_STATE_RUNNING, checking again in 30 seconds...
Job state: JOB_STATE_RUNNING, checking again in 30 seconds...
Job state: JOB_STATE_RUNNING, checking again in 30 seconds...
Job state: JOB_STATE_RUNNING, checking again in 30 seconds...
Job state: JOB_STATE_RUNNING, checking again in 30 seconds...
Job stat

In [30]:
vertex_job.state.name

'JOB_STATE_SUCCEEDED'

In [57]:
minutes, seconds = divmod((vertex_job.end_time - vertex_job.start_time).total_seconds(), 60)
print(f'The job ran for {int(minutes)} minutes and {int(seconds)} seconds')

The job ran for 15 minutes and 16 seconds


### Review The Optimized Result: System Instructions

In [31]:
result = json.loads(
    bucket.blob(f'{SERIES}/{EXPERIMENT}/{job_name}/instruction/optimized_results.json').download_as_string()
)
result

{'step': 5,
 'metrics': {'coherence/mean': 4.8,
  'fluency/mean': 4.8,
  'composite_metric/mean': 4.8},
 'prompt': 'Write a poem of the specified TYPE and TOPIC.  Adhere to the rules and conventions of the given poem TYPE.  The poem should be creative, use vivid imagery, and have a strong rhythm if applicable to the TYPE. Do not generate poems in other formats.\n\nTYPE: Ode\n\nTOPIC: Lego\n\nSpecifically, focus on the elevated and lyrical nature of an ode, praising the qualities of Lego.  Consider its versatility, the joy it brings, and the creativity it inspires.'}

In [32]:
Markdown(result['prompt'])

Write a poem of the specified TYPE and TOPIC.  Adhere to the rules and conventions of the given poem TYPE.  The poem should be creative, use vivid imagery, and have a strong rhythm if applicable to the TYPE. Do not generate poems in other formats.

TYPE: Ode

TOPIC: Lego

Specifically, focus on the elevated and lyrical nature of an ode, praising the qualities of Lego.  Consider its versatility, the joy it brings, and the creativity it inspires.

### Review The Evaluation Process: Step-By-Step

In [35]:
results_ui = prompt_opt.ResultsUI(path = f'gs://{bucket.name}/{SERIES}/{EXPERIMENT}/{job_name}')

In [36]:
results_df_html = """
<style>
  .scrollable {
    width: 100%;
    height: 80px;
    overflow-y: auto;
    overflow-x: hidden;  /* Hide horizontal scrollbar */
  }
  tr:nth-child(odd) {
    background: var(--colab-highlighted-surface-color);
  }
  tr:nth-child(even) {
    background-color: var(--colab-primary-surface-color);
  }
  th {
    background-color: var(--colab-highlighted-surface-color);
  }
</style>
"""

display(HTML(results_df_html))
display(results_ui.get_container())

---
## Optimize Prompts: System Instructions Multi-Shot Examples

### Set Optimization Mode

Use `instruction_and_demo` to optimize the system instructions and include examples, called demonstration, for multi-shot prompting:

In [37]:
OPTIMIZATION_MODE = "instruction_and_demo" # choices are instuction, demonstration, instruction_and_demo

### Run Optimization Job On Vertex AI Training

In [38]:
job_name = 'lego_lyrics_' + datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
vertex_job = prompt_opt.run_apd(
    config = dict(
        project = PROJECT_ID,
        system_instruction = SYSTEM_INSTRUCTIONS,
        prompt_template = PROMPT_TEMPLATE,
        target_model = TARGET_MODEL,
        target_model_location = REGION, 
        eval_metrics_types = EVAL_METRICS,
        eval_metrics_weights = [.5, .5],
        aggregation_type = 'weighted_sum',
        source_model = SOURCE_MODEL,
        optimization_mode = OPTIMIZATION_MODE,
        input_data_path = f'gs://{bucket.name}/{blob.name}',
        output_path = f'gs://{bucket.name}/{SERIES}/{EXPERIMENT}/{job_name}'
    ),
    bucket_uri = f'gs://{bucket.name}/{SERIES}/{EXPERIMENT}/{job_name}',
    display_name = job_name
)



Job display name: lego_lyrics_2025-01-30T12:32:02
Creating CustomJob
CustomJob created. Resource name: projects/1026793852137/locations/us-central1/customJobs/521134942101438464
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/1026793852137/locations/us-central1/customJobs/521134942101438464')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/521134942101438464?project=1026793852137


Check out the Vertex AI Training Job with the link in the output above.

In [39]:
vertex_job.name, vertex_job.display_name, vertex_job.resource_name

('521134942101438464',
 'lego_lyrics_2025-01-30T12:32:02',
 'projects/1026793852137/locations/us-central1/customJobs/521134942101438464')

In [51]:
while vertex_job.state == aiplatform.gapic.JobState.JOB_STATE_PENDING or vertex_job.state == aiplatform.gapic.JobState.JOB_STATE_RUNNING:
    print(f"Job state: {vertex_job.state.name}, checking again in 30 seconds...")
    time.sleep(30)

In [44]:
vertex_job.state.name

'JOB_STATE_SUCCEEDED'

In [56]:
minutes, seconds = divmod((vertex_job.end_time - vertex_job.start_time).total_seconds(), 60)
print(f'The job ran for {int(minutes)} minutes and {int(seconds)} seconds')

The job ran for 15 minutes and 16 seconds


### Review The Optimized Results

#### System Instructions

In [45]:
result = json.loads(
    bucket.blob(f'{SERIES}/{EXPERIMENT}/{job_name}/instruction/optimized_results.json').download_as_string()
)
result

{'step': 2,
 'metrics': {'coherence/mean': 4.8,
  'fluency/mean': 5.0,
  'composite_metric/mean': 4.9},
 'prompt': 'Write a poem with the following specifications:\n\nTYPE: [Specify the type of poem, e.g., Acrostic, Haiku, Limerick, Sonnet, Free Verse, etc.]\n\nTOPIC: [Specify the topic of the poem, e.g., Lego, Nature, Love, Loss, etc.]\n\nSTYLE: [Specify the desired style, e.g., descriptive, narrative, lyrical, humorous, etc.]\n\nTONE: [Specify the desired tone or mood, e.g., joyful, melancholic, reflective, whimsical, etc.]\n\nAUDIENCE: [Specify the intended audience for the poem, e.g., children, adults, a specific person, etc.]\n\nLENGTH: [Specify the desired length, e.g., a specific number of lines, stanzas, or words, or a general range like "short" or "long".]\n\nPURPOSE: [Specify the purpose or message of the poem, e.g., to express a feeling, tell a story, celebrate an event, etc.]\n\nIMAGERY/METAPHORS: [Encourage the use of vivid imagery and metaphors related to the topic.  For 

In [46]:
Markdown(result['prompt'])

Write a poem with the following specifications:

TYPE: [Specify the type of poem, e.g., Acrostic, Haiku, Limerick, Sonnet, Free Verse, etc.]

TOPIC: [Specify the topic of the poem, e.g., Lego, Nature, Love, Loss, etc.]

STYLE: [Specify the desired style, e.g., descriptive, narrative, lyrical, humorous, etc.]

TONE: [Specify the desired tone or mood, e.g., joyful, melancholic, reflective, whimsical, etc.]

AUDIENCE: [Specify the intended audience for the poem, e.g., children, adults, a specific person, etc.]

LENGTH: [Specify the desired length, e.g., a specific number of lines, stanzas, or words, or a general range like "short" or "long".]

PURPOSE: [Specify the purpose or message of the poem, e.g., to express a feeling, tell a story, celebrate an event, etc.]

IMAGERY/METAPHORS: [Encourage the use of vivid imagery and metaphors related to the topic.  For example, if the topic is "Lego," suggest metaphors related to building, creation, imagination, or the specific characteristics of Lego bricks.]

Consider these additional instructions for crafting your poem:

* Use strong verbs and evocative language to create vivid imagery.
* Pay attention to the rhythm and flow of the poem.
* Ensure the poem has a clear structure and progression.
* If applicable, adhere to the specific rules and structure of the chosen poem type (e.g., syllable count for Haiku, rhyming scheme for a Sonnet).


If exemplers/demonstrations are provided in the context, please follow them as guidelines for style, tone, and structure while still adhering to the above specifications.

#### System Instruction With Examples (Demonstration)

In [49]:
result_demonstration = json.loads(
    bucket.blob(f'{SERIES}/{EXPERIMENT}/{job_name}/demonstration/optimized_results.json').download_as_string()
)
result_demonstration

{'step': 6,
 'metrics': {'coherence/mean': 5.0,
  'fluency/mean': 5.0,
  'composite_metric/mean': 5.0},
 'prompt': 'Write a poem with the following specifications:\n\nTYPE: [Specify the type of poem, e.g., Acrostic, Haiku, Limerick, Sonnet, Free Verse, etc.]\n\nTOPIC: [Specify the topic of the poem, e.g., Lego, Nature, Love, Loss, etc.]\n\nSTYLE: [Specify the desired style, e.g., descriptive, narrative, lyrical, humorous, etc.]\n\nTONE: [Specify the desired tone or mood, e.g., joyful, melancholic, reflective, whimsical, etc.]\n\nAUDIENCE: [Specify the intended audience for the poem, e.g., children, adults, a specific person, etc.]\n\nLENGTH: [Specify the desired length, e.g., a specific number of lines, stanzas, or words, or a general range like "short" or "long".]\n\nPURPOSE: [Specify the purpose or message of the poem, e.g., to express a feeling, tell a story, celebrate an event, etc.]\n\nIMAGERY/METAPHORS: [Encourage the use of vivid imagery and metaphors related to the topic.  For 

In [50]:
Markdown(result_demonstration['prompt'])

Write a poem with the following specifications:

TYPE: [Specify the type of poem, e.g., Acrostic, Haiku, Limerick, Sonnet, Free Verse, etc.]

TOPIC: [Specify the topic of the poem, e.g., Lego, Nature, Love, Loss, etc.]

STYLE: [Specify the desired style, e.g., descriptive, narrative, lyrical, humorous, etc.]

TONE: [Specify the desired tone or mood, e.g., joyful, melancholic, reflective, whimsical, etc.]

AUDIENCE: [Specify the intended audience for the poem, e.g., children, adults, a specific person, etc.]

LENGTH: [Specify the desired length, e.g., a specific number of lines, stanzas, or words, or a general range like "short" or "long".]

PURPOSE: [Specify the purpose or message of the poem, e.g., to express a feeling, tell a story, celebrate an event, etc.]

IMAGERY/METAPHORS: [Encourage the use of vivid imagery and metaphors related to the topic.  For example, if the topic is "Lego," suggest metaphors related to building, creation, imagination, or the specific characteristics of Lego bricks.]

Consider these additional instructions for crafting your poem:

* Use strong verbs and evocative language to create vivid imagery.
* Pay attention to the rhythm and flow of the poem.
* Ensure the poem has a clear structure and progression.
* If applicable, adhere to the specific rules and structure of the chosen poem type (e.g., syllable count for Haiku, rhyming scheme for a Sonnet).


If exemplers/demonstrations are provided in the context, please follow them as guidelines for style, tone, and structure while still adhering to the above specifications.
Write a Sonnet about Lego.

The response is: From plastic bricks, a world of dreams takes flight,
A castle grand, a spaceship sleek and bold,
With tiny hands, we build, our stories told,
In vibrant hues, a tapestry of light.
No need for glue, or hammer, or for might,
Just snap and click, a structure to behold,
A dragon fierce, a pirate ship of old,
Each piece a part, a symphony of sight.
From simple squares, to intricate designs,
A boundless realm, where imagination thrives,
A child's delight, a grown-up's fondest wish,
To build and create, where nothing ever dies.
So let us play, with these enduring toys,
And find in Lego, endless, joyful joys. 

==


Write a Acrostic about Lego.

The response is: **L**ittle bricks of plastic,
**E**ndlessly creative,
**G**rowing worlds with each piece,
**O**utstanding imagination. 

==


Write a Limerick about Lego.

The response is: There once was a builder named Sue,
Whose Lego creations were true.
From castles to ships,
With colorful clips,
She built them all, just for you. 


### Review The Evaluation Process: Step-By-Step

In [47]:
results_ui = prompt_opt.ResultsUI(path = f'gs://{bucket.name}/{SERIES}/{EXPERIMENT}/{job_name}')

In [48]:
results_df_html = """
<style>
  .scrollable {
    width: 100%;
    height: 80px;
    overflow-y: auto;
    overflow-x: hidden;  /* Hide horizontal scrollbar */
  }
  tr:nth-child(odd) {
    background: var(--colab-highlighted-surface-color);
  }
  tr:nth-child(even) {
    background-color: var(--colab-primary-surface-color);
  }
  th {
    background-color: var(--colab-highlighted-surface-color);
  }
</style>
"""

display(HTML(results_df_html))
display(results_ui.get_container())